## Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Import Necessary Libraries

In [ ]:
!pip install tabulate
import requests
import pandas as pd
import time
import json

## Scrape posts metadata from OpenAI Developer "Prompt Engineering" forum using Discourse API

In [ ]:
# Step 1: Define file paths
SAVE_PATH = "/content/drive/MyDrive/SW_PROJECT/openai_prompting_posts_metadata.csv"

# Step 2: Define API endpoints
BASE_URL = "https://community.openai.com/c/prompting/8.json"

# Step 3: Set headers to mimic a browser request
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# Step 4: Function to fetch posts (Including Post ID)
def get_filtered_posts():
    """ Fetch posts dynamically, extracting only relevant fields including Post ID """
    all_posts = []
    page = 0
    max_retries = 1  # Retry once for failed pages

    while True:
        print(f"Fetching metadata from page {page}...")

        url = f"{BASE_URL}?page={page}"
        retries = max_retries

        while retries >= 0:
            response = requests.get(url, headers=HEADERS)

            if response.status_code == 200:
                break  # Success, exit retry loop
            else:
                print(f"Failed to fetch page {page}. Retrying... ({retries} retries left)")
                retries -= 1
                time.sleep(3)  # Short delay before retry

        if response.status_code != 200:
            print(f"Skipping page {page}. Status Code: {response.status_code}")
            break  # Exit loop if the API is unresponsive

        data = response.json()
        topics = data.get("topic_list", {}).get("topics", [])

        if not topics:
            print("No more data available.")
            break  # Stop when no more topics are returned

        # Extract only the required metadata fields (including Post ID)
        for post in topics:
            tags = post.get("tags", [])  # Get tags list

            # Assign "prompting" if no tags exist
            if not tags:
                tags = ["prompting"]

            post_data = {
                "Post ID": post["id"],  # Fetch Post ID
                "Title": post["title"],
                "Tags": ", ".join(tags),  # Extract tags as a string
                "Created At": post["created_at"],
                "Last Activity": post["last_posted_at"],
                "Views": post["views"],
                "Replies": post["posts_count"],
                "Participants": len(post.get("posters", []))  # Count of unique users participating
            }
            all_posts.append(post_data)

        # Continue fetching pages
        page += 1
        time.sleep(3)  # Increase pause slightly to avoid rate-limiting

    return pd.DataFrame(all_posts)

# Step 5: Fetch only selected fields dynamically
df_posts = get_filtered_posts()

# Step 6: Convert timestamps to datetime format
df_posts["Created At"] = pd.to_datetime(df_posts["Created At"])
df_posts["Last Activity"] = pd.to_datetime(df_posts["Last Activity"])

# Step 7: Save DataFrame to Google Drive
df_posts.to_csv(SAVE_PATH, index=False)
print(f"Data saved to Google Drive: {SAVE_PATH} with {len(df_posts)} posts!")

Fetching metadata from page 0...
Fetching metadata from page 1...
Fetching metadata from page 2...
Fetching metadata from page 3...
Fetching metadata from page 4...
Fetching metadata from page 5...
Fetching metadata from page 6...
Fetching metadata from page 7...
Fetching metadata from page 8...
Fetching metadata from page 9...
Fetching metadata from page 10...
Fetching metadata from page 11...
Fetching metadata from page 12...
Fetching metadata from page 13...
Fetching metadata from page 14...
Fetching metadata from page 15...
Fetching metadata from page 16...
Fetching metadata from page 17...
Fetching metadata from page 18...
Fetching metadata from page 19...
Fetching metadata from page 20...
Fetching metadata from page 21...
Fetching metadata from page 22...
Fetching metadata from page 23...
Fetching metadata from page 24...
Fetching metadata from page 25...
Fetching metadata from page 26...
Fetching metadata from page 27...
Fetching metadata from page 28...
Fetching metadata from p

In [ ]:
# Load the dataset
file_path = "/content/drive/MyDrive/SW_PROJECT/openai_prompting_posts_metadata.csv"
df = pd.read_csv(file_path)

# Display first few rows
df.head()

,Post ID,Title,Tags,Created At,Last Activity,Views,Replies,Participants
0,36,About the Prompting category,prompting,2021-03-10 20:40:28.668000+00:00,2021-03-10 20:40:28.674000+00:00,27009,1,1
1,1218800,Get my project assistants to talk to each other,assistants-api,2025-04-04 15:29:37.209000+00:00,2025-04-05 08:12:10.218000+00:00,105,4,3
2,889278,"Collection of Dall-E 3 prompting tips, issues ...","dalle3, dalle, dalle3-bugs, gallery",2024-08-01 02:19:18.343000+00:00,2025-04-05 07:15:15.058000+00:00,4668,247,5
3,314682,Politeness in prompt any affect on response?,chatgpt,2023-08-02 16:46:09.115000+00:00,2025-04-05 04:58:38.250000+00:00,6827,12,5
4,1075454,"Assistant referring to ""the files uploaded"" in...",assistants-api,2024-12-30 10:31:31.765000+00:00,2025-04-04 00:17:06.296000+00:00,247,7,5


## Scrape posts main content (Post ID, Title, Post Body) from OpenAI Developer "Prompt Engineering" forum using Discourse API

In [ ]:
# Step 1: Define file paths
metadata_file = "/content/drive/MyDrive/SW_PROJECT/openai_prompting_posts_metadata.csv"
SAVE_PATH_CSV = "/content/drive/MyDrive/SW_PROJECT/openai_prompting_posts.csv"

# Step 2: Define API endpoints
POST_URL = "https://community.openai.com/t/{}.json"

# Step 3: Set headers to mimic a browser request
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# Step 4: Load post metadata
df_posts = pd.read_csv(metadata_file)

# Step 5: Function to fetch post title and body
def get_post_details(post_id):
    url = POST_URL.format(post_id)
    response = requests.get(url, headers=HEADERS)

    if response.status_code == 200:
        data = response.json()

        # Extract main post content
        post_title = data.get("title", "No Title")
        post_body = data["post_stream"]["posts"][0].get("cooked", "No Content")

        return post_title, post_body
    else:
        print(f"Failed to fetch post {post_id}. Status Code: {response.status_code}")
        return None, None

# Step 6: Initialize storage lists
post_details = []
post_count = 0  # Initialize counter

# Step 7: Loop through each post ID
total_posts = len(df_posts)
for index, row in df_posts.iterrows():
    post_id = row["Post ID"]
    post_count += 1  # Increment counter

    print(f"Fetching post {post_count}/{total_posts} (Post ID: {post_id})...")

    post_title, post_body = get_post_details(post_id)

    if post_title is not None:
        # Store only post title and body
        post_data = {
            "Post ID": post_id,
            "Title": post_title,
            "Post Body": post_body
        }
        post_details.append(post_data)

    time.sleep(1)  # Prevent rate-limiting

# Step 8: Convert to DataFrame (Only Post Title & Body)
df_full_posts = pd.DataFrame(post_details)

# Step 9: Save Post Data to CSV
df_full_posts.to_csv(SAVE_PATH_CSV, index=False)

print(f"\nData saved successfully:")
print(f"CSV (Post Title & Body): {SAVE_PATH_CSV} with {len(df_full_posts)} posts!")

Fetching post 1/2693 (Post ID: 36)...
Fetching post 2/2693 (Post ID: 1218800)...
Fetching post 3/2693 (Post ID: 889278)...
Fetching post 4/2693 (Post ID: 314682)...
Fetching post 5/2693 (Post ID: 1075454)...
Fetching post 6/2693 (Post ID: 565091)...
Fetching post 7/2693 (Post ID: 1201440)...
Fetching post 8/2693 (Post ID: 1201889)...
Fetching post 9/2693 (Post ID: 1201666)...
Fetching post 10/2693 (Post ID: 498040)...
Fetching post 11/2693 (Post ID: 4327)...
Fetching post 12/2693 (Post ID: 810313)...
Fetching post 13/2693 (Post ID: 1132524)...
Fetching post 14/2693 (Post ID: 329066)...
Fetching post 15/2693 (Post ID: 1156670)...
Fetching post 16/2693 (Post ID: 1160453)...
Fetching post 17/2693 (Post ID: 1151231)...
Fetching post 18/2693 (Post ID: 1157044)...
Fetching post 19/2693 (Post ID: 1157417)...
Fetching post 20/2693 (Post ID: 1117422)...
Fetching post 21/2693 (Post ID: 597498)...
Fetching post 22/2693 (Post ID: 1130642)...
Fetching post 23/2693 (Post ID: 1154106)...
Fetching pos

In [ ]:
# Load the dataset
file_path = "/content/drive/MyDrive/SW_PROJECT/openai_prompting_posts.csv"
df = pd.read_csv(file_path)

# Display first few rows
df.head()

,Post ID,Title,Post Body
0,36,About the Prompting category,<p>Learn more about prompting by sharing best ...
1,1218800,Get my project assistants to talk to each other,<p>Is it possible to have the defined assistan...
2,889278,"Collection of Dall-E 3 prompting tips, issues ...",<p>Here is a collection of tips and tricks and...
3,314682,Politeness in prompt any affect on response?,<p>Many prompt examples contain human like pol...
4,1075454,"Assistant referring to ""the files uploaded"" in...",<p>Hi! I am using the Assistants API and have ...


## Merging the two datasets

In [ ]:
import pandas as pd

# Load both datasets
meta_path = "/content/drive/MyDrive/SW_PROJECT/openai_prompting_posts_metadata.csv"
content_path = "/content/drive/MyDrive/SW_PROJECT/openai_prompting_posts.csv"

df_meta = pd.read_csv(meta_path)
df_content = pd.read_csv(content_path)

# Drop 'Title' from content file to avoid duplication (metadata already has it)
if 'Title' in df_meta.columns and 'Title' in df_content.columns:
    df_content = df_content.drop(columns=['Title'])

# Merge on 'Post ID'
df_merged = pd.merge(df_meta, df_content, on="Post ID", how="inner")

# Save the merged file
output_path = "/content/drive/MyDrive/SW_PROJECT/openai_prompting_merged.csv"
df_merged.to_csv(output_path, index=False)

print(f"Merged dataset saved to: {output_path}")
print(f"Total merged rows: {len(df_merged)}")

# Load the dataset to verify
df = pd.read_csv(output_path)
df.head()

Merged dataset saved to: /content/drive/MyDrive/SW_PROJECT/openai_prompting_merged.csv
Total merged rows: 2693


,Post ID,Title,Tags,Created At,Last Activity,Views,Replies,Participants,Post Body
0,36,About the Prompting category,prompting,2021-03-10 20:40:28.668000+00:00,2021-03-10 20:40:28.674000+00:00,27009,1,1,<p>Learn more about prompting by sharing best ...
1,1218800,Get my project assistants to talk to each other,assistants-api,2025-04-04 15:29:37.209000+00:00,2025-04-05 08:12:10.218000+00:00,105,4,3,<p>Is it possible to have the defined assistan...
2,889278,"Collection of Dall-E 3 prompting tips, issues ...","dalle3, dalle, dalle3-bugs, gallery",2024-08-01 02:19:18.343000+00:00,2025-04-05 07:15:15.058000+00:00,4668,247,5,<p>Here is a collection of tips and tricks and...
3,314682,Politeness in prompt any affect on response?,chatgpt,2023-08-02 16:46:09.115000+00:00,2025-04-05 04:58:38.250000+00:00,6827,12,5,<p>Many prompt examples contain human like pol...
4,1075454,"Assistant referring to ""the files uploaded"" in...",assistants-api,2024-12-30 10:31:31.765000+00:00,2025-04-04 00:17:06.296000+00:00,247,7,5,<p>Hi! I am using the Assistants API and have ...
